# Transfer Learning Example

### Problem Statement

In this tutorial, we shall improve on the exisiting classifier but taking the Transfer Learning approach.

We shall use the <b>VGG16 CNN Architecture from Keras Framework pre-trained on ImageNet Dataset</b>.

We shall follow the similar steps as before with a few changes in defining the model:
    1. Load the Data CIFAR10
    2. Load the pre-trained VGG16 model from Keras and Freeze the first few layers
    3. Add some end layers to cater for Classifying 10 Classes as in CIFAR10
    4. Train the model
    5. Evaluate the model on Test set
 

In [1]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

/Users/prateethvnayak/Documents/Carlson Talk/CSOM-Computer-Vision/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/prateethvnayak/Documents/Carlson Talk/CSOM-Computer-Vision/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/prateethvnayak/Documents/Carlson Talk/CSOM-Computer-Vision/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '

# Load the Data

In [2]:
# Load Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Mention the Class Name List
class_list = ['airplane','automobile','bird','cat','deer','dog',
              'frog','horse','ship','truck']

# # Convert a one-hot vector for the test-labels
y_train = keras.utils.to_categorical(y_train, num_classes=len(class_list))


# # Split the test set to Validation & Test set
num_test_samples = x_test.shape[0]
x_val, y_val = x_test[0:num_test_samples//2,:,:], y_test[0:num_test_samples//2]
x_test, y_test = x_test[num_test_samples//2:,:,:], y_test[num_test_samples//2:]

y_val = keras.utils.to_categorical(y_val, num_classes=len(class_list))

# Load the Model

In [6]:
from keras.models import Model
from keras.applications import vgg16 as vgg
from keras.layers import Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
# Get the pre-trained model
vgg_base_model = vgg.VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(32,32,3))

# Get the Last output from the base model from last layer
last_output = vgg_base_model.get_layer('block3_pool').output

# Add new Layers
x = GlobalAveragePooling2D()(last_output)
x = BatchNormalization()(x)
x = Dense(256, activation='relu', name='d1')(x)
x = Dense(256, activation='relu', name='d2')(x)

pred = Dense(len(class_list),activation='softmax')(x)

# Build the model
model = Model(inputs=vgg_base_model.input, outputs=pred)

# Freeze layers of Base Model
for layer in vgg_base_model.layers:
    layer.trainable = False
        
# Compile the Model with the Loss function, Optimizer and Accuracy Metric
model.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=1e-4),metrics=['accuracy'])

# Visualize the model
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [ ]:
x_train, y_train = shuffle(x_train, y_train)
# Train the model
model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=7)

Train on 50000 samples, validate on 5000 samples
Epoch 1/7
 1696/50000 [>.............................] - ETA: 5:21 - loss: 2.3946 - accuracy: 0.1356

In [ ]:
# Predict on Test Set
y_pred = np.argmax(model.predict(x_test), axis=-1).reshape(5000,1)

acc = np.sum(y_test == y_pred) / y_test.shape[0]
# Print Accuracy
print("Test Accuracy :{}".format(acc*100))